In [1]:
import shutil
import os
from Libraries.grid_bboxes_library import * 
from Libraries.lungs_detection_library import *

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


C:\Users\marcp\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\_tensor.py:1142: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\BatchLinearAlgebra.cpp:859.)
  ret = func(*args, **kwargs)


Get the lungs detector model 

In [2]:
lungsDetectorModel = setupLungsDetectionLearner()

Get the original dataset of pleural effusion/normal images with their full size 

In [3]:
def getImgs(setName, labelType): #setName can be 'train'/'valid'/'test'; labelType can be 'normal'/'effusion'
    data_dir = os.path.join(original_ds_path, setName, labelType)
    imgs = os.listdir(data_dir)
    imgs = [os.path.join(data_dir, f) for f in imgs if f.endswith('.png')]
    return imgs

In [4]:
original_ds_path = '../Datasets/RepoGithub_ref/original_ds/' 
grids_ds_path = '../Datasets/RepoGithub_ref/grids_ds'

assert os.path.isdir(original_ds_path), "Couldn't find the dataset at {}".format(original_ds_path.data_dir)

# Get the images paths of each set (train/valid/test) and type (normal/effusion)
imgsPaths = {'train': (getImgs('train', 'normal'), getImgs('train', 'effusion')),
                 'valid': (getImgs('valid', 'normal'), getImgs('valid', 'effusion')),
                 'test': (getImgs('test', 'normal'), getImgs('test', 'effusion'))}

Extract the grids of the detected lungs where pleural effusion can be located and create a new dataset with these grids

In [5]:
def extractPleuralGrids(imgPath):
    """Return the grid of the left lung and the grid of the right lung where pleural effusion can be located"""
    
    imgOri = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE) 

    imgRes = cv2.imread(imgPath) 
    
    predictedBboxes = lungsDetectorModel.predict(imgPath) # get a TensorBBox of size 2x4 with the bboxes coordinates
    ''' Example of predictedBboxes:
    (TensorBBox([[  7.9246,  13.2136,  74.8705, 100.0172],
        [ 78.8447,   8.7797, 146.1627,  99.2884]]), TensorBase([-0.9009, -0.7798, -0.0641,  0.6670, -0.0144, -0.8537,  0.8270,  0.6548]), 
        TensorBase([-0.9009, -0.7798, -0.0641,  0.6670, -0.0144, -0.8537,  0.8270,  0.6548]))
    '''
    # example of predictedBboxes: TensorBBox([[  7.9246,  13.2136,  74.8705, 100.0172], [ 78.8447,   8.7797, 146.1627,  99.2884]])
    predictedBboxes = ((predictedBboxes[1] + 1) / 2).numpy()


    leftLung = predictedBboxes[0:4] * [imgOri.shape[0], imgOri.shape[1], imgOri.shape[0], imgOri.shape[1]]
    # convert a TensorBBox to a numpy array, so that, we can access each coord as a flot
    rightLung = predictedBboxes[4:8] * [imgOri.shape[0], imgOri.shape[1], imgOri.shape[0], imgOri.shape[1]]

    gridsLungLeft = gridBbox(leftLung, imgRes, imgOri, 3, 3) # grid bbox left in the X-ray image / right in the body
    gridsLungRight = gridBbox(rightLung, imgRes, imgOri, 3, 3) # grid bbox right

    return gridsLungLeft[6], gridsLungRight[8]

In [6]:
size = 64

def saveGridsResized(gridLeftLung, gridRightLung, output_dir):
    """Resize the grids of an image, so that all grids have the same size, and save them to the `output_dir`"""

    # Use bilinear interpolation instead of the default "nearest neighbor" method
    gridLeftLung = cv2.resize(gridLeftLung, (size, size), interpolation = cv2.INTER_AREA)
    gridRightLung = cv2.resize(gridRightLung, (size, size), interpolation = cv2.INTER_AREA)
    # Save each grid
    cv2.imwrite(os.path.join(output_dir,'D3.png'), gridLeftLung)
    cv2.imwrite(os.path.join(output_dir,'E3_flipped.png'), gridRightLung)

Main

In [7]:
# Process train, valid and test sets
grids_train_val_dir = os.path.join(grids_ds_path, "train_val") 
grids_test_dir = os.path.join(grids_ds_path, "test") 

for set in ['train', 'valid', 'test']:
    # Get the path of 'train'/'valid'/'test' sets on the new dataset (grids_ds)
    # train and valid sets will be stored inside 'train_val' folder 
    if set == 'test':
        output_dir_set = grids_test_dir
    else:
        output_dir_set = os.path.join(grids_train_val_dir, set) 

    # Get the paths of 'normal' and 'effusion' subfolders 
    normal_dir = os.path.join(output_dir_set, 'normal')
    effusion_dir = os.path.join(output_dir_set, 'effusion')

    # Delete if possible the folders 'normal' and 'effusion' from previous executions of this notebook
    shutil.rmtree(normal_dir, ignore_errors=True) # making ignore_errors = True will not raise a FileNotFoundError in case directory doesn't exist
    shutil.rmtree(effusion_dir, ignore_errors=True)

    # Create folders named 'normal' and 'effusion' and their parent folders named 'train'/'valid'/'test'
    os.makedirs(normal_dir, exist_ok=True)
    os.makedirs(effusion_dir, exist_ok=True)

    print("Processing {} data, saving grids to {}".format(set, output_dir_set))
    # Process the images of each set (train/valid/test)
    # Each set has the same number of normal and "pleural effusion" images  
    # Process normal imgs first and then process the effusion imgs
    for i in range(2): 
        grand_parent_grids_dir = [normal_dir, effusion_dir] 
        for j in range(len(imgsPaths[set][i])): # imgsPaths[set][0] are the normal images paths of a set; imgsPaths[set][1] are the effusion images paths of a set
            # Process each normal/effusion image from the corresponding set (train/val/test)
            imgPath = imgsPaths[set][i][j] 
            imgName = imgPath.split('\\')[-1] # for example, if imgPath = '[...] train\normal\normal1.png'; imgName = normal1.png 
            output_grids_dir = os.path.join(grand_parent_grids_dir[i], imgName[:-4]) # output_grids_dir has the name of an image without it's extension (.png)
            gridLeftLung, gridRightLung = extractPleuralGrids(imgPath)
            # Flip one of the grids horizontally to make both grids more similar to each other, so that the training of the effusion model performs better
            gridRightLung =  cv2.flip(gridRightLung, 1) # 1 = horizontal flipping; # 0 = vertical flipping; -1 = vertical + horizontal flipping
            os.makedirs(output_grids_dir, exist_ok=True)
            saveGridsResized(gridLeftLung, gridRightLung, output_grids_dir)
   
print("Done building dataset")

Processing train data, saving grids to ../Datasets/RepoGithub_ref/grids_ds\train_val\train


Processing valid data, saving grids to ../Datasets/RepoGithub_ref/grids_ds\train_val\valid


Processing test data, saving grids to ../Datasets/RepoGithub_ref/grids_ds\test


Done building dataset
